In [76]:
import nba_api
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonallplayers
import pandas as pd

In [14]:

nba_players = players.get_players() #list of dicts, each dict is an nba player
active_players = [player for player in nba_players if player['is_active']]

In [23]:
combinestats = pd.read_csv('DraftCombineStats.csv', encoding='ISO-8859-1') #csv of draft combine stats (wingspan, height, weight, etc.)
combinestats

,PLAYER,POS,BODY FAT %,HAND LENGTH (INCHES),HAND WIDTH (INCHES),HEIGHT W/O SHOES,HEIGHT W/ SHOES,STANDING REACH,WEIGHT (LBS),WINGSPAN
0,Ndongo N'diaye,C,-%,-,-,6' 11.5'',NaN,9' 4.5'',208,7' 3.5''
1,Karim Shabazz,C,-%,-,-,6' 11.5'',NaN,9' 3.5'',224,7' 3''
2,Primoz Brezec,C,-%,-,-,7' 0.75'',NaN,9' 2'',243,7' 2''
3,Jameel Watkins,PF-C,-%,-,-,6' 9.5'',NaN,9' 2'',244.5,7' 6''
4,Ernest Brown,C,-%,-,-,6' 10'',NaN,9' 1.5'',222.5,7' 1.25''
...,...,...,...,...,...,...,...,...,...,...
1628,James Nnaji,C,-%,-,-,NaN,NaN,NaN,-,NaN
1629,Brice Sensabaugh,SF,-%,-,-,NaN,NaN,NaN,-,NaN
1630,Nick Smith Jr.,SG,-%,-,-,NaN,NaN,NaN,-,NaN
1631,Nae'Qwan Tomlin,PF,-%,-,-,NaN,NaN,NaN,-,NaN


In [61]:
allplayers = commonallplayers.CommonAllPlayers().get_data_frames()[0]
allplayers = allplayers[['PERSON_ID', 'DISPLAY_FIRST_LAST']]
allplayers.head()

,PERSON_ID,DISPLAY_FIRST_LAST
0,76001,Alaa Abdelnaby
1,76002,Zaid Abdul-Aziz
2,76003,Kareem Abdul-Jabbar
3,51,Mahmoud Abdul-Rauf
4,1505,Tariq Abdul-Wahad


In [62]:
merged_df = pd.merge(combinestats, allplayers, how = 'left', left_on = 'PLAYER', right_on='DISPLAY_FIRST_LAST') #merge the 2 dfs on player id
merged_df = merged_df.dropna(subset = ['PERSON_ID']) #drop the players that aren't in allplayers -> this means they haven't played in nba yet
merged_df = merged_df.drop(columns= ['DISPLAY_FIRST_LAST']) #drop this col
merged_df['PERSON_ID'] = merged_df['PERSON_ID'].astype(int)
merged_df

,PLAYER,POS,BODY FAT %,HAND LENGTH (INCHES),HAND WIDTH (INCHES),HEIGHT W/O SHOES,HEIGHT W/ SHOES,STANDING REACH,WEIGHT (LBS),WINGSPAN,PERSON_ID
2,Primoz Brezec,C,-%,-,-,7' 0.75'',NaN,9' 2'',243,7' 2'',2056
4,Ernest Brown,C,-%,-,-,6' 10'',NaN,9' 1.5'',222.5,7' 1.25'',2081
5,Jabari Smith,PF-C,-%,-,-,6' 9.75'',NaN,9' 1.5'',240.5,7' 2.5'',2074
6,Malik Allen,PF-C,-%,-,-,6' 8.25'',NaN,9' 1'',271,7' 2.5'',2124
11,Dan McClintock,C-PF,-%,-,-,6' 11'',NaN,8' 11'',250,7' 1.5'',2082
...,...,...,...,...,...,...,...,...,...,...,...
1638,Jett Howard,SF,-%,-,-,NaN,NaN,NaN,-,NaN,1641724
1642,Brandon Miller,SF,-%,-,-,NaN,NaN,NaN,-,NaN,1641706
1644,Brice Sensabaugh,SF,-%,-,-,NaN,NaN,NaN,-,NaN,1641729
1645,Nick Smith Jr.,SG,-%,-,-,NaN,NaN,NaN,-,NaN,1641733


In [64]:
#now that we have the df with combine stats and player id, we can now add career stats to the column too
# Iterate through each row in the dataframe
for index, row in merged_df.iterrows():
    player_id = row['PERSON_ID']
    try:
        # Fetch career statistics for the current player
        career_stats = playercareerstats.PlayerCareerStats(player_id=player_id, per_mode36='Per36')
        career_stats_data = career_stats.get_data_frames()[0]

        #if pts has nans then don't append
        if not career_stats_data['PTS'].isna().any():

            # Append career stats columns to the current row
            for column in career_stats_data.columns:
                # Use loc to access and modify the dataframe
                merged_df.loc[index, column] = career_stats_data.at[0, column]

    except Exception as e:
        print(merged_df.loc[index]['PLAYER'])
        print(f"Error fetching career stats for player_id {player_id}: {e}")

Alex Scales
Error fetching career stats for player_id 2637: 0
Herbert Hill
Error fetching career stats for player_id 201195: 0
Brian Butch
Error fetching career stats for player_id 202221: 0
Robert Vaden
Error fetching career stats for player_id 201987: 0
Terrico White
Error fetching career stats for player_id 202358: 0
Justin Lewis
Error fetching career stats for player_id 1631171: 0
Jaylen Clark
Error fetching career stats for player_id 1641740: 0
Isaiah Wong
Error fetching career stats for player_id 1631209: 0
Adam Flagler
Error fetching career stats for player_id 1641745: 0


Now we have a dataframe with the per 36 stats for each of these players along with their draft measurements!

In [78]:
merged_df.head() 

,PLAYER,POS,BODY FAT %,HAND LENGTH (INCHES),HAND WIDTH (INCHES),HEIGHT W/O SHOES,HEIGHT W/ SHOES,STANDING REACH,WEIGHT (LBS),WINGSPAN,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
2,Primoz Brezec,C,-%,-,-,7' 0.75'',NaN,9' 2'',243,7' 2'',...,0.600,3.6,2.7,6.3,1.3,0.0,1.6,1.3,6.3,9.6
4,Ernest Brown,C,-%,-,-,6' 10'',NaN,9' 1.5'',222.5,7' 1.25'',...,0.250,1.7,8.4,10.1,0.0,0.0,1.7,5.0,6.7,5.0
5,Jabari Smith,PF-C,-%,-,-,6' 9.75'',NaN,9' 1.5'',240.5,7' 2.5'',...,0.667,0.5,3.8,4.4,3.3,2.2,0.0,1.6,3.8,14.2
6,Malik Allen,PF-C,-%,-,-,6' 8.25'',NaN,9' 1'',271,7' 2.5'',...,0.800,3.4,5.1,8.5,1.1,0.7,1.8,0.4,3.6,11.6
11,Dan McClintock,C-PF,-%,-,-,6' 11'',NaN,8' 11'',250,7' 1.5'',...,0.000,6.2,4.3,10.5,0.6,0.0,1.2,1.8,6.8,11.1


In [83]:
df = pd.DataFrame({'A': 1, 'B': 2})
csv_file_path = 'test.csv'
try:
    df.to_csv(csv_file_path, index=False)
    print(f"DataFrame 'merged_df' has been saved to {csv_file_path}")
except Exception as e:
    print(f"Error: {e}")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\nyc8p\AppData\Local\Temp\ipykernel_12816\1096372519.py", line 1, in <module>
    df = pd.DataFrame({'A': 1, 'B': 2})
  File "c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\pandas\core\frame.py", line 709, in __init__
    "Use public APIs instead.",
  File "c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\pandas\core\internals\construction.py", line 481, in dict_to_mgr
    arrays = [com.maybe_iterable_to_list(data[k]) for k in keys]
  File "c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\pandas\core\internals\construction.py", line 115, in arrays_to_mgr
    else:
  File "c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\pandas\core\internals\construction.py", line 645, in _extract_index
ValueError: 

In [79]:
merged_df.to_csv('MeasurementsandStats.csv', index = False)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\nyc8p\AppData\Local\Temp\ipykernel_12816\4105981963.py", line 1, in <module>
    merged_df.to_csv('MeasurementsandStats.csv', index = False)
  File "c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\pandas\core\generic.py", line 3772, in to_csv
    header: bool_t | list[str] = ...,
  File "c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\pandas\io\formats\format.py", line 1159, in to_csv
    )
  File "c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\pandas\io\formats\csvs.py", line 24, in <module>
    from pandas._typing import SequenceNotStr
ImportError: cannot import name 'SequenceNotStr' from 'pandas._typing' (c:\Users\nyc8p\anaconda3\envs\tensorflowshit\lib\site-packages\pandas\_typing.py)

During handling of t